In [ ]:
import math
from scipy.stats import norm
import numpy as np


In [4]:
import math
from scipy.stats import norm
import numpy as np
import yfinance as yf

def get_options_data(ticker, expiration_date):
  """
  Retrieves options data for a given ticker and expiration date from Yahoo Finance.

  Args:
    ticker: The stock ticker symbol (e.g., "AAPL").
    expiration_date: The expiration date of the options (e.g., "2024-01-19").

  Returns:
    A pandas DataFrame containing the options data, or None if an error occurs.
  """
  try:
    ticker_object = yf.Ticker(ticker)
    options = ticker_object.option_chain(expiration_date)
    calls = options.calls
    puts = options.puts
    return calls, puts  # Return both calls and puts dataframes

  except Exception as e:
    print(f"An error occurred: {e}")
    return None

# Example usage
ticker_symbol = "AAPL"  # Replace with the desired ticker
expiration = "2024-12-06"  # Replace with desired expiration date

calls_df, puts_df = get_options_data(ticker_symbol, expiration)

if calls_df is not None and puts_df is not None:
    print("Calls Data:")
    print(calls_df.head())  # Print first few rows of calls data
    print("\nPuts Data:")
    print(puts_df.head())  # Print first few rows of puts data

Calls Data:
        contractSymbol             lastTradeDate  strike  lastPrice  bid  ask  \
0  AAPL241206C00130000 2024-11-12 20:40:07+00:00   130.0      94.58  0.0  0.0   
1  AAPL241206C00135000 2024-11-04 20:50:00+00:00   135.0      87.35  0.0  0.0   
2  AAPL241206C00140000 2024-11-14 16:44:41+00:00   140.0      87.43  0.0  0.0   
3  AAPL241206C00145000 2024-11-04 18:45:12+00:00   145.0      77.97  0.0  0.0   
4  AAPL241206C00150000 2024-11-15 15:10:00+00:00   150.0      75.38  0.0  0.0   

   change  percentChange  volume  openInterest  impliedVolatility  inTheMoney  \
0     0.0            0.0     NaN             1            0.00001        True   
1     0.0            0.0     NaN             1            0.00001        True   
2     0.0            0.0     1.0             5            0.00001        True   
3     0.0            0.0     NaN            10            0.00001        True   
4     0.0            0.0     1.0             2            0.00001        True   

  contractSize

#Binomial Option Pricing Model

The simplest method to price the options is to use a binomial option pricing model. This model uses the assumption of perfectly efficient markets. Under this assumption, the model can price the option at each point of a specified time frame.

Under the binomial model, we consider that the price of the underlying asset will either go up or down in the period. Given the possible prices of the underlying asset and the strike price of an option, we can calculate the payoff of the option under these scenarios, then discount these payoffs and find the value of that option as of today.

In [6]:
def binomial_option_pricing(S, K, T, r, u, d, n, option_type="call"):
    """
    Calculate the price of an option using the Binomial Option Pricing Model.

    S: Initial stock price
    K: Strike price
    T: Time to expiration (in years)
    r: Risk-free interest rate
    u: Upward factor
    d: Downward factor
    n: Number of periods
    option_type: 'call' or 'put'
    """
    dt = T / n  # time step
    q = (math.exp(r * dt) - d) / (u - d)  # risk-neutral probability
    disc = math.exp(-r * T)  # discount factor

    # Compute stock prices at maturity
    stock_prices = [S * (u ** j) * (d ** (n - j)) for j in range(n + 1)]

    # Option payoffs at maturity
    if option_type == "call":
        payoffs = [max(0, price - K) for price in stock_prices]
    elif option_type == "put":
        payoffs = [max(0, K - price) for price in stock_prices]

    # Discount back to present value
    for i in range(n):
        payoffs = [(q * payoffs[j + 1] + (1 - q) * payoffs[j]) for j in range(len(payoffs) - 1)]

    return payoffs[0]


#Black-Scholes Model

The Black-Scholes model was developed mainly for pricing European options on stocks. The model operates under certain assumptions regarding the distribution of the stock price and the economic environment. The assumptions about the stock price distribution include:

Continuously compounded returns on the stock are normally distributed and independent over time.
The volatility of continuously compounded returns is known and constant.
Future dividends are known (as a dollar amount or as a fixed dividend yield).
The assumptions about the economic environment are:

The risk-free rate is known and constant.
There are no transaction costs or taxes.
It is possible to short-sell with no cost and to borrow at the risk-free rate.

In [7]:
def black_scholes(S, K, T, r, sigma, option_type="call"):
    """
    Calculate the price of a European option using the Black-Scholes Model.

    S: Current stock price
    K: Strike price
    T: Time to maturity (in years)
    r: Risk-free interest rate
    sigma: Volatility of the stock
    option_type: 'call' or 'put'
    """
    d1 = (math.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * math.sqrt(T))
    d2 = d1 - sigma * math.sqrt(T)

    if option_type == "call":
        price = S * norm.cdf(d1) - K * math.exp(-r * T) * norm.cdf(d2)
    elif option_type == "put":
        price = K * math.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    return price


#Monte-Carlo Simulation

Monte-Carlo simulation is another option pricing model we will consider. The Monte-Carlo simulation is a more sophisticated method to value options. In this method, we simulate the possible future stock prices and then use them to find the discounted expected option payoffs.

In [8]:
def monte_carlo_simulation(S, K, T, r, sigma, num_simulations=10000, option_type="call"):
    """
    Calculate the price of an option using Monte Carlo simulation.

    S: Current stock price
    K: Strike price
    T: Time to expiration (in years)
    r: Risk-free interest rate
    sigma: Volatility
    num_simulations: Number of simulations
    option_type: 'call' or 'put'
    """
    np.random.seed(42)
    payoff_sum = 0
    for _ in range(num_simulations):
        z = np.random.normal(0, 1)  # standard normal random variable
        ST = S * math.exp((r - 0.5 * sigma ** 2) * T + sigma * math.sqrt(T) * z)
        if option_type == "call":
            payoff = max(0, ST - K)
        elif option_type == "put":
            payoff = max(0, K - ST)
        payoff_sum += payoff

    return math.exp(-r * T) * (payoff_sum / num_simulations)


In [9]:
# Inputs
S = 100  # Current stock price
K = 110  # Strike price
T = 1    # Time to expiration (1 year)
r = 0.05  # Risk-free rate (5%)
sigma = 0.2  # Volatility (20%)
u = 1.1  # Upward factor
d = 0.9  # Downward factor
n = 2  # Number of periods

# Binomial Option Pricing
binomial_price = binomial_option_pricing(S, K, T, r, u, d, n, option_type="call")
print("Binomial Option Price:", binomial_price)

# Black-Scholes Pricing
bs_price = black_scholes(S, K, T, r, sigma, option_type="call")
print("Black-Scholes Option Price:", bs_price)

# Monte-Carlo Simulation
mc_price = monte_carlo_simulation(S, K, T, r, sigma, num_simulations=100000, option_type="call")
print("Monte-Carlo Option Price:", mc_price)


Binomial Option Price: 4.318566843814337
Black-Scholes Option Price: 6.040088129724239
Monte-Carlo Option Price: 6.060288324860743


**Conclusions:**

The three models (Binomial, Black-Scholes, and Monte Carlo) provide different estimates for the option price.
The differences arise from their underlying assumptions and methodologies.
The Binomial model offers a discrete-time approach, the Black-Scholes model relies on continuous-time assumptions and normally distributed returns and the Monte-Carlo simulation utilizes random sampling to estimate price.
In practice, the choice of the appropriate model depends on the specific characteristics of the underlying asset and the option contract, and factors such as transaction costs and market liquidity that are often ignored by these models.
The Monte Carlo simulation typically provides a more accurate estimate, particularly when the underlying asset's price dynamics are complex, but it requires more computational resources.